In [1]:
import pandas as pd
import numpy as np

In [11]:
data = pd.read_csv('train.csv', encoding='latin-1')


In [12]:
data.head()

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [14]:
# Assuming 'df' is your DataFrame
columns_to_drop = ['textID', 'selected_text', 'Time of Tweet','Age of User','Country','Population -2020','Land Area (Km²)','Density (P/Km²)']

# Drop the specified columns
new_data = data.drop(columns=columns_to_drop)

In [6]:
new_data

,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative
...,...,...
27476,wish we could come see u on Denver husband l...,negative
27477,I`ve wondered about rake to. The client has ...,negative
27478,Yay good for both of you. Enjoy the break - y...,positive
27479,But it was worth it ****.,positive


In [16]:
new_data = new_data[new_data.sentiment != "neutral"]
new_data.head()

,text,sentiment
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative
6,2am feedings for the baby are fun when he is a...,positive


In [18]:
new_data['sentiment'].unique()

array(['negative', 'positive'], dtype=object)

In [19]:
def remove_non_alphanumeric(text):
    cleaned_text = ''
    for char in text:
        if char.isalnum() or char.isspace():
            cleaned_text += char
    return cleaned_text

In [20]:
new_data['text'] = new_data['text'].apply(lambda x: x.lower())
new_data['text'] = new_data['text'].apply(remove_non_alphanumeric)

In [21]:
new_data

,text,sentiment
1,sooo sad i will miss you here in san diego,negative
2,my boss is bullying me,negative
3,what interview leave me alone,negative
4,sons of why couldnt they put them on the rel...,negative
6,2am feedings for the baby are fun when he is a...,positive
...,...,...
27475,enjoy ur night,positive
27476,wish we could come see u on denver husband l...,negative
27477,ive wondered about rake to the client has ma...,negative
27478,yay good for both of you enjoy the break you...,positive


In [23]:
print(data[ data['sentiment'] == 'positive'].size) #Positive
print(data[ data['sentiment'] == 'negative'].size) #Negative

85820
77810


### The reason for removing 'rt' might be to clean the text data, as 'rt' often appears in tweets to indicate a retweet. Removing it can be useful for certain types of text analysis or natural language processing tasks to focus on the core content of the text.

In [24]:
for idx,row in new_data.iterrows():
    row[0] = row[0].replace('rt',' ')

C:\Users\HP\AppData\Local\Temp\ipykernel_20204\3383244808.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[0] = row[0].replace('rt',' ')
C:\Users\HP\AppData\Local\Temp\ipykernel_20204\3383244808.py:2: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[0] = row[0].replace('rt',' ')


In [27]:
from keras.preprocessing.text import Tokenizer
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(new_data['text'].values)

In [32]:
from keras.preprocessing.sequence import pad_sequences
X = tokenizer.texts_to_sequences(new_data['text'].values)
X = pad_sequences(X)
print(X)

[[   0    0    0 ...   98   11 1401]
 [   0    0    0 ... 1328    9   15]
 [   0    0    0 ...  371   15  440]
 ...
 [   0    0    0 ...  756    4   77]
 [   0    0    0 ...  213  589  742]
 [   0    0    0 ...   26  573    8]]


In [36]:
X[90].shape

(32,)

In [38]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

embed_dim = 128
lstm_out = 196

model = Sequential()
'''the Embedding layer is transforming each word in your text into a 128-dimensional vector. 
This transformation is learned during the training of your model. 
The output of the Embedding layer is a 2D vector with the length of the input sentence and the embedding dimension (128 in your case) 
for each word in the sentence.'''
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))

model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 128)           256000    
                                                                 
 spatial_dropout1d (Spatial  (None, 32, 128)           0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 511194 (1.95 MB)
Trainable params: 511194 (1.95 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [45]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

Y = pd.get_dummies(new_data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(11454, 32) (11454, 2)
(4909, 32) (4909, 2)


In [46]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 40, batch_size=batch_size, verbose = 2)

Epoch 1/40
358/358 - 36s - loss: 0.4420 - accuracy: 0.7808 - 36s/epoch - 100ms/step
Epoch 2/40
358/358 - 33s - loss: 0.2882 - accuracy: 0.8820 - 33s/epoch - 91ms/step
Epoch 3/40
358/358 - 28s - loss: 0.2477 - accuracy: 0.9011 - 28s/epoch - 77ms/step
Epoch 4/40
358/358 - 28s - loss: 0.2163 - accuracy: 0.9123 - 28s/epoch - 77ms/step
Epoch 5/40
358/358 - 28s - loss: 0.1896 - accuracy: 0.9268 - 28s/epoch - 78ms/step
Epoch 6/40
358/358 - 28s - loss: 0.1706 - accuracy: 0.9340 - 28s/epoch - 78ms/step
Epoch 7/40
358/358 - 28s - loss: 0.1522 - accuracy: 0.9406 - 28s/epoch - 77ms/step
Epoch 8/40
358/358 - 28s - loss: 0.1342 - accuracy: 0.9479 - 28s/epoch - 77ms/step
Epoch 9/40
358/358 - 28s - loss: 0.1234 - accuracy: 0.9512 - 28s/epoch - 77ms/step
Epoch 10/40
358/358 - 28s - loss: 0.1094 - accuracy: 0.9577 - 28s/epoch - 77ms/step
Epoch 11/40
358/358 - 28s - loss: 0.1044 - accuracy: 0.9606 - 28s/epoch - 78ms/step
Epoch 12/40
358/358 - 28s - loss: 0.0899 - accuracy: 0.9667 - 28s/epoch - 78ms/step


# Training the model using Pytorch 

In [48]:


import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

# Check if GPU is available and if not, use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Print whether GPU or CPU is being used
print('Using', 'GPU' if device.type == 'cuda' else 'CPU')
class LSTMNet(nn.Module):
    def __init__(self):
        super(LSTMNet, self).__init__()
        self.embed = nn.Embedding(max_fatures, embed_dim)
        self.dropout = nn.Dropout(0.4)
        self.lstm = nn.LSTM(embed_dim, lstm_out, batch_first=True, dropout=0.2)
        self.dense = nn.Linear(lstm_out, 2)
        
    def forward(self, x):
        x = self.embed(x)
        x = self.dropout(x)
        lstm_out, _ = self.lstm(x)
        x = self.dense(lstm_out[:, -1, :])
        return F.log_softmax(x, dim=1)

# Convert your data to PyTorch tensors
X_train_torch = torch.tensor(X_train, dtype=torch.long).to(device)
Y_train_torch = torch.tensor(Y_train, dtype=torch.float).to(device)

X_test_torch = torch.tensor(X_test, dtype=torch.long).to(device)
Y_test_torch = torch.tensor(Y_test, dtype=torch.float).to(device)

# Create TensorDatasets for training and testing
train_data = TensorDataset(X_train_torch, Y_train_torch)
test_data = TensorDataset(X_test_torch, Y_test_torch)

# Create DataLoaders for training and testing
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

# Initialize the model, optimizer, and loss function
py_model = LSTMNet().to(device)
optimizer = torch.optim.Adam(py_model.parameters())
loss_fn = nn.CrossEntropyLoss()

# Training loop
for epoch in range(40):
    for inputs, labels in train_loader:
        py_model.zero_grad()
        output = py_model(inputs)
        loss = loss_fn(output, torch.max(labels, 1)[1])
        loss.backward()
        optimizer.step()
    print(f'Epoch: {epoch+1}/40.............', end=' ')
    print(f'Loss: {loss.item()}')


Using GPU
Epoch: 1/40............. Loss: 0.44195064902305603
Epoch: 2/40............. Loss: 0.2798157334327698
Epoch: 3/40............. Loss: 0.33801567554473877
Epoch: 4/40............. Loss: 0.2740136981010437
Epoch: 5/40............. Loss: 0.2281075119972229
Epoch: 6/40............. Loss: 0.2732827067375183
Epoch: 7/40............. Loss: 0.10638570040464401
Epoch: 8/40............. Loss: 0.42052140831947327
Epoch: 9/40............. Loss: 0.07918696850538254
Epoch: 10/40............. Loss: 0.09327013790607452
Epoch: 11/40............. Loss: 0.08769256621599197
Epoch: 12/40............. Loss: 0.10565336793661118
Epoch: 13/40............. Loss: 0.08638720214366913
Epoch: 14/40............. Loss: 0.06775138527154922
Epoch: 15/40............. Loss: 0.0324229896068573
Epoch: 16/40............. Loss: 0.09184589982032776
Epoch: 17/40............. Loss: 0.20899057388305664
Epoch: 18/40............. Loss: 0.10351333767175674
Epoch: 19/40............. Loss: 0.08520432561635971
Epoch: 20/40....

In [49]:
# Switch model to evaluation mode
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        output = model(inputs)
        # sum up batch loss
        test_loss += loss_fn(output, torch.max(labels, 1)[1]).item()
        # get the index of the max log-probability
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(torch.max(labels, 1)[1].view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))



Test set: Average loss: 0.0248, Accuracy: 4230/4909 (86%)



In [57]:
import re

# Your sentence
sentence = "Recession Hit veronique branquinho, she has to quit her company, such a shame!"
sentence = sentence.lower()
sentence = re.sub('[^a-zA-z0-9\s]','',sentence)
print(sentence)
# Tokenize the sentence
sentence = tokenizer.texts_to_sequences(sentence)

# Pad the sequence
sentence = pad_sequences(sentence, maxlen=X.shape[1])

# Convert to tensor
sentence = torch.tensor(sentence, dtype=torch.long).to(device)

# Get the model's prediction
prediction = py_model(sentence)

# Print the prediction
if torch.argmax(prediction) == 1:
    print("Positive sentiment")
else:
    print("Negative sentiment")


recession hit veronique branquinho she has to quit her company such a shame
Negative sentiment


In [58]:
# Save the model
torch.save(model.state_dict(), 'sentiment.pth')